<h1> Plus proche voisin </h1>

<h2> Présentation </h2>

On place aléatoirement n points sur une droite orientée.
Ces points $P_i$ sont classés par abscisse croissante sur la droite.
Soit $P_j$ le plus proche voisin du point $P_i$ ($P_j$ peut être le point $P_{i-1}$ ou le point $P_{i+1}$). Quelle est la probabilité que le plus proche voisin de $P_j$ soit $P_i$ ?

Même question pour des points sur un plan (on prendra la norme euclidienne).

Même question pour des points dans un espace à 3 dimensions.

<h2> Solution : </h2>

Les valeurs théoriques sont :
<ol>
<li> à 1 D : $\frac 23\simeq$ 0.666
<li> à 2 D : $\frac{6\pi}{8.\pi+3.\sqrt{3}}\simeq 0.6215$
<li> à 3 D : $\frac{16}{27}\simeq 0.5926$
</ol>

In [1]:
#use "topfind";;
#require "plplot";;
open Plplot;;
module P = Plot;;
let couleurs_list = [[ 0;255;255;255]; (*`white*)
                     [ 1;  0;  0;  0]; (*`black*)
                     [ 2;  0;  0;255]; (*`blue*)
                     [ 3;255;  0;  0]; (*`red*)
                     [ 4;165; 42; 42]; (*`brown*)
                     [ 5;  0;  0;  0]; [ 6;  0;  0;  0]; [ 7;  0;  0;  0]; [ 8;  0;  0;  0]; [ 9;  0;  0;  0]; 
                     [10;200;200;200]; (*`gray*)
                     [11;  0;255;255]; (*`light_blue*)
                     [12;  0;255;  0]; (*`green*)
                     [13;255;255;  0]; (*`yellow*)
                     [14;255;  0;255]; (*`pink*)
                     [15;160;  0;213]; (*`purple*) ]
let rec loop couleurs_list = match couleurs_list with
    | [n;r;g;b]::tl -> plscol0 n r g b; loop tl
    | _ -> ();;
let couleurs = (fun () -> plscolbg 255 255 255; loop couleurs_list)
let initialisation filename xmin xmax ymin ymax = 
        P.init (xmin, ymin) (xmax, ymax) `greedy (`svg `core) ~filename:(filename^".svg") ~pre:couleurs
let xlabel texte = P.text_outside `black (`bottom 0.5) 3. texte
let ylabel texte = P.text_outside `black (`left 0.5) 5. texte 
let label texte_x texte_y titre = P.label texte_x texte_y titre

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/usr/lib/ocaml/plplot: added to search path
/usr/lib/ocaml/plplot/plplot.cma: loaded


module P = Plplot.Plot

val couleurs_list : int list list =
 [[0; 255; 255; 255]; [1; 0; 0; 0]; [2; 0; 0; 255]; [3; 255; 0; 0];
 [4; 165; 42; 42]; [5; 0; 0; 0]; [6; 0; 0; 0]; [7; 0; 0; 0]; [8; 0; 0; 0];
 [9; 0; 0; 0]; [10; 200; 200; 200]; [11; 0; 255; 255]; [12; 0; 255; 0];
 [13; 255; 255; 0]; [14; 255; 0; 255]; [15; 160; 0; 213]]

val loop : int list list -> unit = <fun>

val couleurs : unit -> unit = <fun>

val initialisation : string -> float -> float -> float -> float -> P.stream_t =
 <fun>

val xlabel : string -> P.plot_t = <fun>

val ylabel : string -> P.plot_t = <fun>

val label : string -> string -> string -> P.plot_t = <fun>

In [2]:
let range debut fin step =
   let rec range i acc =
     if i>=fin then List.rev acc
     else range (i+step) (i::acc) in
   range debut [];; 

val range : int -> int -> int -> int list = <fun>

In [3]:
open Random;;
Random.self_init;;

let voisins_1D nbre_points =
    let abscisse = Array.init nbre_points (fun _ -> Random.float 1.) in
    Array.sort compare abscisse;

    let voisin = Array.init nbre_points
                 (fun j -> if j=0 then 1 else if j=nbre_points-1 then nbre_points-2
                           else if abscisse.(j)-.abscisse.(j-1)<abscisse.(j+1)-.abscisse.(j) then j-1
                           else j+1) in

    let rec loop j nbre_appariement=
        if j<nbre_points-1 then
            if voisin.(j)=j+1 && voisin.(j+1)=j 
            then loop (j+2) (nbre_appariement+1)
            else loop (j+1) nbre_appariement
        else float_of_int (2*nbre_appariement) /.(float_of_int nbre_points) in
    loop 1 (if voisin.(1)=0 then 1 else 0);;

let plus_proche_2D num_point abscisse ordonnee =
    let nbre_points = Array.length abscisse in
    let x = abscisse.(num_point) in
    let y = ordonnee.(num_point) in
    let rec loop num_voisin distance_min plus_proche =
         if num_voisin = num_point
         then loop (num_voisin+1) distance_min plus_proche
         else if num_voisin < nbre_points then begin
              let d=(abscisse.(num_voisin)-.x)**2.+.(ordonnee.(num_voisin)-.y)**2. in
              if d<distance_min 
              then loop (num_voisin+1) d num_voisin
              else loop (num_voisin+1) distance_min plus_proche;
              end
        else plus_proche in
    loop 0 max_float 0;;

let plus_proche_3D num_point abscisse ordonnee altitude =
    let nbre_points = Array.length abscisse in
    let x = abscisse.(num_point) in
    let y = ordonnee.(num_point) in
    let z = altitude.(num_point) in
    let rec loop num_voisin distance_min plus_proche =
         if num_voisin = num_point
         then loop (num_voisin+1) distance_min plus_proche
         else if num_voisin < nbre_points then begin
              let d=(abscisse.(num_voisin)-.x)**2.+.(ordonnee.(num_voisin)-.y)**2.+.(altitude.(num_voisin)-.z)**2. in
              if d<distance_min 
              then loop (num_voisin+1) d num_voisin
              else loop (num_voisin+1) distance_min plus_proche;
              end
        else plus_proche in
    loop 0 max_float 0;;

let voisins_2D nbre_points =
    let abscisse = Array.init nbre_points (fun _ -> Random.float 1.) in
    let ordonnee = Array.init nbre_points (fun _ -> Random.float 1.) in

    let voisin = Array.init nbre_points (fun num_point -> plus_proche_2D num_point abscisse ordonnee) in

    let rec loop num_point nbre_appariement =
        if num_point<nbre_points then begin
            let plus_proche = voisin.(num_point) in
            if voisin.(plus_proche) = num_point
            then loop (num_point+1) (nbre_appariement+1)
            else loop (num_point+1) nbre_appariement
            end
        else float_of_int nbre_appariement/.(float_of_int nbre_points) in
    loop 0 0;;

let voisins_3D nbre_points =
    let abscisse = Array.init nbre_points (fun _ -> Random.float 1.) in
    let ordonnee = Array.init nbre_points (fun _ -> Random.float 1.) in
    let altitude = Array.init nbre_points (fun _ -> Random.float 1.) in
        
    let voisin = Array.init nbre_points (fun num_point -> plus_proche_3D num_point abscisse ordonnee altitude) in

    let rec loop num_point nbre_appariement =
        if num_point<nbre_points then begin
            let plus_proche = voisin.(num_point) in
            if voisin.(plus_proche) = num_point
            then loop (num_point+1) (nbre_appariement+1)
            else loop (num_point+1) nbre_appariement
            end
        else float_of_int nbre_appariement/.(float_of_int nbre_points) in
    loop 0 0;;

let essai nbre_points fonction_voisin =
    let nbre_essais = 1000 in
    let rec loop num_essai res =
        if num_essai<nbre_essais
        then loop (num_essai+1) (res+.(fonction_voisin nbre_points))
        else res/.(float_of_int nbre_essais) in
    loop 0 0.;;
    
let liste_nbre_points = range 4 40 1 in
let xs = Array.of_list (List.map float_of_int liste_nbre_points) in
let ys = Array.of_list (List.map (fun x -> essai x voisins_1D) liste_nbre_points) in
let ys' = Array.of_list (List.map (fun x -> essai x voisins_2D) liste_nbre_points) in
let ys'' = Array.of_list (List.map (fun x -> essai x voisins_3D) liste_nbre_points) in

let p = initialisation "graph" 0. 40. 0.56 0.68 in
P.plot ~stream:p [P.lines `red xs ys;
                  P.lines `green xs ys';
                  P.lines `blue xs ys'';
                  P.legend [[P.line_legend "1D" `red];
                            [P.line_legend "2D" `green];
                            [P.line_legend "3D" `blue]]];
P.finish ~stream:p ();;

- : unit -> unit = <fun>

val voisins_1D : int -> float = <fun>

val plus_proche_2D : int -> float array -> float array -> int = <fun>

val plus_proche_3D : int -> float array -> float array -> float array -> int =
 <fun>

val voisins_2D : int -> float = <fun>

val voisins_3D : int -> float = <fun>

val essai : 'a -> ('a -> float) -> float = <fun>

- : unit = ()

<img src="graph.svg" width=500 />